In [4]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#chargement fichiers
dfa_full = pd.read_csv("AllMoviesDetailsCleaned.csv", sep=";", low_memory=False, parse_dates=["release_date"])
dfb_full = pd.read_csv("AllMoviesCastingRaw.csv", sep=";", low_memory=False)

In [5]:
#Fusion fichiers par ids communes
df_full = pd.merge(dfa_full, dfb_full, on='id')
#Création fichier avec les seules données pertinentes
df_rel = df_full[['id','director_name','release_date']].sort_values(by='director_name',ascending=True)


id                        int64
director_name            object
release_date     datetime64[ns]
dtype: object

In [6]:
df_rel = df_rel.fillna('0')
df_rel['release_date']=df_rel['release_date'].map(lambda x:x.year)
#On réindexe
df_rel2 = df_rel.reset_index(drop = True)

In [10]:
#calcul du temps moyen entre films
df = df_rel2
#valeurs retenues pour chaque réal: nombre de films, liste des années de sortie des films
dir_time={}
x = 0
#parcours de la liste de réals
while x < len(df):
    dir_name = df.loc[x,'director_name']
    dir_time[dir_name] = {'movie_nb':0, 'dates':[]}
    #les réals sont par ordre alphabétique, on parcout la liste de films de chacun
    while df.loc[x,'director_name'] == dir_name :
        dir_time[dir_name]['movie_nb'] +=1
        #on ajoute la date à la liste si elle n'y est pas déjà
        if df.loc[x,'release_date'] not in dir_time[dir_name]['dates']:
            dir_time[dir_name]['dates'].append(df.loc[x,'release_date'])
        x += 1
        if x == len(df):
            break
#print(x)
#print(dir_time)

dir_avg = {}
for key in dir_time:
    # temps entre films: valeur de 2 ans par défaut
    if dir_time[key]['movie_nb'] == 1:
            dir_avg[key] = 2.0
    #sinon: 
    else:
        min_date = min(dir_time[key]['dates'])
        max_date = max(dir_time[key]['dates'])
        movie_nb = dir_time[key]['movie_nb']
        #valeur par défaut de 2 ans pour le premier film
        dir_avg[key] = round((max_date - min_date + 2) / movie_nb, 1)

print(dir_avg)


{'\tCheung Chi-Sing': 2.5, '\tRaymond Fung Sai-Hung': 2.0, ' \tHuang Jian-Zhong': 4.4000000000000004, ' \tZhang Jing': 1.7, '    Faris Kermani ': 2.0, ' Anna van Keimpema': 2.5, ' Barnaby Southcombe': 2.0, ' Ben Caron': 2.0, ' Dale Trevillion\t ': 2.7999999999999998, ' Davide Manuli': 8.0, ' E.W. Swackhamer': 1.3, ' Henrik Georgsson': 2.0, ' Huang Chien-Liang': 2.0, ' Hugues Martin': 2.0, ' Jeon Hyung-jun': 2.0, ' Joy Fleury ': 3.5, ' Lim Dae-Woong': 2.0, ' Nelson Venkatesan': 2.0, ' P Sagaya Suresh': 2.0, ' Sean J. Vincent': 2.0, ' Thanya Phowijit': 2.0, ' Tôru Hosokawa': 3.0, ' Vitaliy Vorobyov': 1.3999999999999999, ' Yeon Sang-Ho': 2.0, ' Yoon Jae-goo': 4.0, '%20': 1.5, "'Evil' Ted Smith": 2.0, '300ml': 6.5, '4608': 2.0, '50 Cent': 2.0, 'A K Dolven': 2.0, 'A K Sajan': 5.5, 'A Mohan Gandhi': 2.0, 'A R Badul': 2.5, 'A Ragavendra Prasad': 2.0, 'A S Ravikumar Chowdhary': 2.0, 'A. B. Mughan': 2.0, 'A. Barr-Smith': 1.5, 'A. Barrere': 2.0, 'A. Bhaskar Rao': 2.0, 'A. Bhimsingh': 1.0, 'A. Bl

In [11]:
with open('dir_avgtime.csv','w',encoding='utf-8') as file:
    writer = csv.writer(file, delimiter=';', lineterminator='\n')
    writer.writerow(('director_name', 'avg_time'))
    for director_name in dir_avg:
        row=[director_name,dir_avg[director_name]]
        writer.writerow(row)
